In [1]:
import streamlit as st
import pandas as pd
import re
import nltk
nltk.download('wordnet')
from nltk import WordNetLemmatizer
from difflib import get_close_matches
import streamlit as st

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rajeev.mishra\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# Load the data
uom = pd.read_csv("uom.csv")
location = pd.read_csv("indianLocationList.csv", encoding="ISO-8859-1")
# prepositions = pd.read_csv("prepositions_updated.csv")['prepositions'].tolist()
prepositions = pd.read_csv("Place_preposition_Product.csv")['Preposition'].tolist()
shortCodes = pd.read_csv("shortCodesProduct.csv")
procurement = pd.read_csv("procurementTerms.csv")
product_df = pd.read_csv("Updated_keywordProductSynonym2.csv", encoding = "Windows-1252")
product_df['synonymkeyword'] = product_df['synonymkeyword'].fillna('')
company_df = pd.read_csv("Copy of company_list_with_abbr.csv")
stop_words = pd.read_csv("stopwords_updated.csv")['0'].tolist()


In [11]:
stop_words = pd.read_csv("stopwords_updated.csv")['0'].tolist()

In [12]:
stop_words

['0o',
 'medium',
 'and',
 '0s',
 '3a',
 '3b',
 'large',
 '3d',
 'small',
 '6b',
 '6o',
 'a',
 'a1',
 'a2',
 'a3',
 'a4',
 'ab',
 'able',
 'about',
 'above',
 'abst',
 'ac',
 'accordance',
 'according',
 'accordingly',
 'across',
 'act',
 'actually',
 'ad',
 'added',
 'adj',
 'ae',
 'af',
 'affected',
 'affecting',
 'affects',
 'after',
 'afterwards',
 'ag',
 'again',
 'against',
 'ah',
 'ain',
 "ain't",
 'aj',
 'al',
 'all',
 'allow',
 'allows',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'amoungst',
 'amount',
 'an',
 'and',
 'announce',
 'another',
 'any',
 'anybody',
 'anyhow',
 'anymore',
 'anyone',
 'anything',
 'anyway',
 'anyways',
 'anywhere',
 'ao',
 'ap',
 'apart',
 'apparently',
 'appear',
 'appreciate',
 'appropriate',
 'approximately',
 'ar',
 'are',
 'aren',
 'arent',
 "aren't",
 'arise',
 'around',
 'as',
 "a's",
 'aside',
 'ask',
 'asking',
 'associated',
 'at',
 'au',
 'auth',
 'av',
 'available',
 'aw',
 '

In [13]:
def search_keywords(input_text2):
    words = input_text2.split()
    cleaned_words = []
    for i in range(len(words)):
        if words[i].lower() in prepositions:
            break
        cleaned_words.append(words[i])
    output_text = ' '.join(cleaned_words)
    print("output_text", output_text)
    # remove unwanted characters
    output_text = output_text.replace(",", " BRK").replace(".", " BRK")
    output_text = re.findall(r'[a-zA-Z]+', output_text)
    # remove stop words
    filtered_words = [word for word in output_text if word.lower() not in stop_words]
    print("filtered_words", filtered_words)
    # initialize variables
    keyword_matches = []
    remaining_words = filtered_words
    wordNotFound=[]
    # search for longest possible matching word strings in keyword column
    while len(remaining_words) > 0:
        for i in range(len(remaining_words), 0, -1):
            phrase = ' '.join(remaining_words[:i])
            matches = product_df[product_df['keyword'].str.lower() == phrase.lower()]
            if len(matches) > 0:
                keyword_matches.append((matches.iloc[0]['keycodeid'], phrase))
                remaining_words = remaining_words[i:]
                break
        else:
            # no match found in keyword column, try synonym column
            for i in range(len(remaining_words), 0, -1):
                phrase = ' '.join(remaining_words[:i])
                matches = product_df[product_df['synonymkeyword'].str.lower()==phrase.lower()]
                if len(matches) > 0:
                    keyword_matches.append((matches.iloc[0]['synonymId'], phrase))
                    remaining_words = remaining_words[i:]
                    break
            else:
                # no match found in synonym column, try productname column
                for i in range(len(remaining_words), 0, -1):
                    phrase = ' '.join(remaining_words[:i])
                    matches = product_df[product_df['ProductName'].str.lower()==phrase.lower()]
                    if len(matches) > 0:
                        keyword_matches.append((matches.iloc[0]['ProductCode'], phrase))
                        remaining_words = remaining_words[i:]
                        break
                else:
                    # no match found in any column
                    wordNotFound.append(remaining_words.pop(0))
    # return keycodeids and corresponding phrases
    return keyword_matches, wordNotFound

In [38]:
def final(input_text):

    lemm = WordNetLemmatizer()
    # spell = SpellChecker()
    wordL = []
    close_matches =[]
    A = search_keywords(input_text)
    # print("A:==>",A)

    not_found_words = A[1]
    for word in not_found_words:
        wordlemmatized = lemm.lemmatize(word)
        wordL.append(wordlemmatized)
    wordL = ' '.join(wordL)
    B = search_keywords(wordL)
    # print("B:==>",B)

    wordforCloseMatching = B[1]
    # print("B(1)", B[1])
    getCloseMatch = []
    for i in wordforCloseMatching:
        # print(i)
        c = get_close_matches(i, product_df['keyword'], n=5)
        print("c", c)
        # temp=[]
        # for i in c:
        #     if i[0].lower()==c[i][0]:
        #         temp.append(c[i])
        first_letter = i[0].lower()
        print("first_letter", first_letter)
        matches = [match for match in c if match.lower().startswith(first_letter)]
        print("matches", matches)
        # print('temp value:',temp)
        getCloseMatch.append(matches)
        for items in getCloseMatch:
            getCloseMatch = [items for items in getCloseMatch if items is not None]
    # print("getCloseMatch",getCloseMatch)    
    # getCloseMatch = ' '.join(getCloseMatch)
    result = []
    for l in getCloseMatch:
        result += l
    # print(result)
    brahmastra = ' '.join(result)
    # print("brahmastra", brahmastra)
    D = search_keywords(brahmastra)

    code_A = A[0]
    code_B = B[0]
    code_D = D[0]

    return code_A + code_B + code_D

In [40]:
input_text = "Supply Of Closed System Ns Iv Infusion 100 Ml, Closed System Ns Iv Infusion 500ml, Closed System D5 Iv Infusion 500 Ml, Close System Dns Iv Infusion 500 Ml, Four Lumen Central Venous Catheter Line With Ecg Lead , Triple Lumen Central Venous Catheter Line With Ecg Lead , Epidural Set-Adult, Total Parenteral Nutrion 625ml, Total Parenteral Nutrion 1250ml Peri, Examination Gloves, Urine Collection Bag 2000ml, Foley Catheter Size 12 To 16 Fg, Paediatric Drip Set, Oxygen Mask With Reservoir - Adult, Cannula Fixator, Bifurcated Small Bore Extension Set Twin Port With Positive Displacement, Nebulizer Mask, Wound Closure Suction Set All Size, Intravenous Cannula Size 18g To 22g, Intravenous Cannula 16g, Polypropylene Mesh Lp Blue 3 Inch X 15 Cm , Polypropylene Mesh Lp Blue 6 Inch X15 Cm , Polypropylene Mesh Elastic Blue 30 X 30 Cm , Disposable Skin Stapler , Et Tube Cuffed , Pmo Line , Adult Ventilator Circuit With Hme Filter And Catheter Mount, Re-Breathing Bag Adult Or Child, Spinal Needle , Vented Iv Set With Y Port Dehp Free Latex Free, Three-Way Stopcock, Micro Drip Set, Advance Blood Transfusion Set With 200ym-11cm2 Particle Filter With Prime Stop, Suction Catheter From Size 10 To 16 Fg, Extension Line 10 To 50 Cm, Extension Line 100 To 200 Cm, Oropharyngeal Airway Or Guedel Airway All Size, Nasopharyngeal Airway All Size, Oxygen Mask Adult, Nasal Cannula Adult, Nelaton Catheter All Size, Silicon Foley Catheter Size- 12 Fg To 24 Fg, Chest Drainage Catheter, Trocar Catheter All Size, Yankaur Suction Set, Ryles Tube All Size, Feeding Tube 5 To 10 Fg, Alcohol Swab, Blood Lancet, Electro Cautery Pencil, Bain Circuit Adult, Tracheostomy Tube Cuffed From 7 To 8 Fg, Syringes 2 Ml With Needle Ribbon Pack , Syringes 3 Ml With Needle Ribbon Pack , Syringes 5 Ml With Needle Ribbon Pack , Syringes 10 Ml With Needle Ribbon Pack , Syringes 20 Ml With Needle Ribbon Pack , Syringes 50 Ml Without Needle Ribbon Pack , Pop Bandage 15cm, Elastic Adhesive Bandage , Orthopedic Cast Bandage , Crepe Bandage 10 Cm X 4 Mtr, Crepe Bandage Size 15 Cm X 4 Mtr, C-Arm Cover, Disposable Surgeon Cap , Disposable Shoe Cover, Medical Disposable Surgeon Gown, Swab Stick, Surgical Blade Size 10 To 15 And 20 To 25, Hemoglobin Cuvettes, Surgical Gloves Powdered From Size 6.5 To 8 As Per Buyer , Surgical Gloves Powder Free Size 6.5 To 8 As Per Buyer, Ecg Electrode, Wastage Bag Black, Absorbable Hemostats Dressing Size 5 X 5 Cm, Absorbable Hemostats Dressing Size 5 X 10 Cm, Absorbable Hemostats Dressing Size 10 X 10 Cm, Absorbable Hemostats Dressing Size 8 X 20 Cm, Absorbable Hemostats Dressing Size 8 X 100 Cm, Absorbable Hemostats Dressing Size 20 X 30 Cm, Adhesive Non-Woven Dressing With Absorbent Pad, Plastic Plain Sheet For Ot , Cotton 500gm, Paper Tape 3 Inch , Catheter Mount, Antimicrobial Incise Drape , Chlorhexidine Gauze Dressing Size 10cm X 10cm, Chlorhexidine Gauze Dressing Size 30cm X 10cm, T-Piece, Bone Wax, Fully Plasma Adapted Balanced Crystalloid Solution With Calcium Acetate And Maleate, Disposable Bed Sheet, Dextrose 25 Percent, Plastic Surgical Tape, Combine Dressing Pad 10 X 10 Sterilized, Dressing Pad 20 X 10 Sterilized, Fiberglass Cast 6 Inch, Fiberglass Cast 4 Inch, Gauze Than F-Ii 90cm X 18m, Lens Cleaning Solution, Lens Cleaning Paper, Filter Paper, Leishman Stain Solution, Plain Glass Slide, Immersion Oil, Micro Tips, Tourniquet, Esr Tube, Lp Needle, Esr Rack, Round Adhesive Band-Aid, Vacutainer Needle For Blood Collection, Bio Medical Waste Bag, Polyamide Sutures Blue 2-0 75cm , Polyamide Sutures Blue 3-0 75cm , Polyglactin Sutures 910 Violet 2-0 90cm , Polyglactin Sutures 910 Violet 1 90cm , Polyglactin Sutures 910 Violet 3-0 70cm , Polyglactin Sutures 910 Violet 0 90cm , Polyglactin Suture 910 Violet 4-0 70cm, Polypropylene Sutures 3-0 2 75cm , Polypropylene Sutures 4-0 2 75cm , Polypropylene Sutures 5-0 75cm , Silk Black Sutures 0 76cm , Silk Black Sutures 2-0 75cm , Polydioxanone Violet 1 150cm Loop Pds Ii No 1 On Loop, Polypropylene Sutures 2-0 2 75cm Hr30 , Poly-4-Hydroxybutylate Suture Size 1, Polyglactin Sutures 910 Violet 4-0 70cm Hr17, Polyglactin Suture 910 5-0 70cm Hr17 , Polypropylene Suture 6-0 75cm, Polypropylene Suture 7-0 60cm, Silk Black Suture 3-0 75cm , Silk Black Suture 4-0 75cm , Silk Black Suture 2-0 75cm , Hand Sanitizer 500ml, Three Ply Face Mask, Blankets For Patient, Patient Dress, Fogging Machine, Micro Centrifuge Tube, Skin Marker, Povidone Iodine 5 Percent, Povidone Iodine 7.5 Percent, Povidone Iodine 10 Percent, Sevoflurane, Isoflurane, Absorbable Gelatin Sponge, Bed Traction, Cervical Traction Kit, Foot-Skin Traction Kit, Tissue Paper, Hand Wash Liquid, White Phenyl, Surgical Cap Female, Pop Bandage 10 Cm, Abdominal Mop Sponge, Sleeper, Bleaching Powder, Glucose Test Strip, Wipes, Surgical Dressing 12.5 Cm X 12.5 Cm, Surgical Dressing 10 X 20, Surgical Dressing 7.5 X 7.5, Surgical Dressing 22.5 X 22.5, Surgical Dressing 10 X 10, Knee O Drape, Hip U Drape, Viral Barrier Kit, Cling Drape, Enternal Nutrition Powder, Hme Filter, Skin Grafting Blade, Disposable Skin Prep Blade, Sterilit Power Systems Oil Spray, Edta Vacuum Blood Collection Tube, Fluoride Vacuum Blood Collection Tube, Plain Vacuum Blood Collection Tube, Sst Vacuum Blood Collection Tube, Sodium Citrate Vacuum Blood Collection Tube."

In [41]:
final(input_text)

output_text Supply Of Closed System Ns Iv Infusion 100 Ml, Closed System Ns Iv Infusion 500ml, Closed System D5 Iv Infusion 500 Ml, Close System Dns Iv Infusion 500 Ml, Four Lumen Central Venous Catheter Line With Ecg Lead , Triple Lumen Central Venous Catheter Line With Ecg Lead , Epidural Set-Adult, Total Parenteral Nutrion 625ml, Total Parenteral Nutrion 1250ml Peri, Examination Gloves, Urine Collection Bag 2000ml, Foley Catheter Size 12 To 16 Fg, Paediatric Drip Set, Oxygen Mask With Reservoir - Adult, Cannula Fixator, Bifurcated Small Bore Extension Set Twin Port With Positive Displacement, Nebulizer Mask, Wound Closure Suction Set All Size, Intravenous Cannula Size 18g To 22g, Intravenous Cannula 16g, Polypropylene Mesh Lp Blue 3 Inch X 15 Cm , Polypropylene Mesh Lp Blue 6 Inch X15 Cm , Polypropylene Mesh Elastic Blue 30 X 30 Cm , Disposable Skin Stapler , Et Tube Cuffed , Pmo Line , Adult Ventilator Circuit With Hme Filter And Catheter Mount, Re-Breathing Bag Adult Or Child, Spi